Download relevant data from the AREG model as provided by the Adriatic forecasting system ([AFS](http://oceanlab.cmcc.it/afs/)). Our simulations only cover the period of February to September so we will only download data for these months (01-09) for the years 2004-2018.

Next field will download all files - will take a while - be patient.

__Disclaimer:__ Since the beginning of 2020 the AFS is not operational any more and has stopped maintaining the servers which hosted the AREG data originally, so the code for downloading the data below does not work any more. However, the raw data used for the simulations below can still be obtained from the CMCC (download link will be provide on request. contact: operation_ocean-lab@cmcc.it). Alternatively, please contact Christoph Hahn (christoph.hahn@uni-graz.at) if would like to reproduce the analalyses (as described in Sefc et al. 2020). We have all raw data backed up and are happy to provide it on request.

In [ ]:
%%bash
#this is where the AFS has currently deposited their data
baseurl='http://tds.tessa.cmcc.it/thredds/fileServer/AFS/simulation/day'
startyear=2004
endyear=2018
startmonth=01
endmonth=09

for year in $(seq $startyear $endyear)
do
    echo -e "$year"
    mkdir $year
    
    #the following is just a complicated way to download only files for the range of months specified
    #The first loop formats the month number with a leading zero if necessary
    #the second loop downloads the list of files per year and greps only the ones for the current month
    #the third one does the downloading
    for l in $(for j in $(for i in $(seq $startmonth $endmonth)
    do
        echo $i
    done | perl -ne 'chomp; print sprintf("%02d",$_)."\n"')
    do
        wget -qO- http://tds.tessa.cmcc.it/thredds/catalog/AFS/simulation/day/$year/catalog.html | grep "simu.nc.gz" | grep -P "$j.._areg"
    done | sed 's/ /|/g')
    do
        #this part does the actual downloading
        filename=$(echo -e "$l" | cut -d "'" -f 2 | perl -ne 'chomp; @a=split("/"); print "$a[-1]\n"')
        echo "downloading $baseurl/$year/$filename -> $year/$filename"
        wget -qO $year/$filename $baseurl/$year/$filename
    done

#    #decompress files
#    for file in $(find ./$year/ -name "*.gz")
#    do
#        gunzip -v $file
#    done
done

Import packages.

In [ ]:
%matplotlib inline
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile
import numpy as np
import math
from datetime import timedelta
from operator import attrgetter

Specify target populations.

In [ ]:
target = [
    'Rovinj',
    'Valsaline',
    'Sarkodana',
    'Cres',
    'Krk_neu',
    'Krk_Baska',
    'Klenovica',
    'Rab',
    'HafenPag_neu',
    'Pag',
    'PagerBucht',
    'Pag_Holger',
    'DugiOtok',
    'Ugljan_neu',
    'Pasman',
    'SibenikDolac_neu',
    'Kornaten_Sued',
    'TrogirCiovoWest',
    'BracSupetarSutivan',
    'BracPovlja',
    'CampAloaBol',
    'HvarBucht4',
    'HvarMlaska',
    'HvarBogomoje',
    'Mljet',
    'Molunat_neu'
]

Read in lat lon coordinates from file.

In [ ]:
fh = open('coordinates_MAX.csv')

fh.readline()

populations = []
region = []
start_lon = []
start_lat = []

for line in fh.readlines():
    line = line.strip()
#    print (line)
    if line.split(",")[0] in target:
        populations.append(line.split(",")[1])
        region.append(line.split(",")[2])
        start_lon.append(line.split(",")[-2])
        start_lat.append(line.split(",")[-1])
    
#print (populations, region, start_lon, start_lat)

Now do the simulations year by year.

## 2004

First decompress all files and specify some parameters.

In [ ]:
%%bash

year=2004
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2004
#specify start/end date for simulation
start = '2004-02-01'
end = '2004-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

Define fieldset for parcels.

In [ ]:
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

check time data.

In [ ]:

#define start time/date and end time/date
start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)


Define the particles.

In [ ]:
### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    

Check the particles.

Visualize the starting points.

In [ ]:
pset[0].show(field=fieldset.U)

Run the simulations.

In [ ]:
for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    


Have a look at some results.

In [ ]:
file="./2004/release-2004-02-01T00-00-00.000000000.nc"
plotTrajectoriesFile(file,
                     mode='2d')

In [ ]:
plotTrajectoriesFile(file,
                     tracerfile='./2004/20040217_areg2c_simu.nc',
                     tracerlon='lon_u',
                     tracerlat='lat_v',
                     tracerfield='depth')

cleanup.

In [ ]:
del(pset)

In [ ]:
%%bash

year=2004
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done


# 2005

In [ ]:
%%bash

year=2005
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

Run all from above in one cell.

In [ ]:
year = 2005
#specify start/end date for simulation
start = '2005-02-01'
end = '2005-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2005
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2006

In [ ]:
%%bash

year=2006
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2006
#specify start/end date for simulation
start = '2006-02-01'
end = '2006-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2006
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2007

In [ ]:
%%bash

year=2007
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2007
#specify start/end date for simulation
start = '2007-02-01'
end = '2007-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2007
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2008

In [ ]:
%%bash

year=2008
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2008
#specify start/end date for simulation
start = '2008-02-01'
end = '2008-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2008
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2009

In [ ]:
%%bash

year=2009
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2009
#specify start/end date for simulation
start = '2009-02-01'
end = '2009-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2009
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2010

In [ ]:
%%bash

year=2010
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2010
#specify start/end date for simulation
start = '2010-02-01'
end = '2010-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2010
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2011

In [ ]:
%%bash

year=2011
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2011
#specify start/end date for simulation
start = '2011-02-01'
end = '2011-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2011
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2012

In [ ]:
%%bash

year=2012
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2012
#specify start/end date for simulation
start = '2012-02-01'
end = '2012-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2012
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2013

In [ ]:
%%bash

year=2013

#in 2013 there is a switch from areg2d to areg2c and there is one week for which two versions are available
#the below code just decompresses the data for weeks that are unique and if duplicated decompresses only 
#one of the files. Otherwise parcels is not going to accept the fieldset.

#decompress only weeks for which only a single file is available
for valid in $(ls -1 ./$year/*.gz | cut -d "_" -f 1 |sort -n | uniq -c | grep "^.*1 " | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
    gunzip -v $valid*
done

#for dates for which more than one file are available decompress only one
for duplicate in $(ls -1 ./$year/*.gz | cut -d "_" -f 1 |sort -n | uniq -c | grep -v "^.*1 " | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
    gunzip -v $(ls -1 $duplicate* | sort -n | head -n 1)
done

In [ ]:
year = 2013
#specify start/end date for simulation
start = '2013-02-01'
end = '2013-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2013
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2014

In [ ]:
%%bash

year=2014
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2014
#specify start/end date for simulation
start = '2014-02-01'
end = '2014-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2014
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2015

In [ ]:
%%bash

year=2015

#in year 2015 there was a switch from 0.25 degrees resolution to 0.125 degrees resolution, so for some weeks 
#there are two files. we'll only decompress the one with 0.125 degrees resolution otherwise parcels will not
#accept the fieldset.

#decompress only weeks for which only a single file is available
for valid in $(ls -1 ./$year/*.gz | cut -d "_" -f 1 |sort -n | uniq -c | grep "^.*1 " | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
    gunzip -v $valid*
done

#for dates for which more than one file are available choose only one
for duplicate in $(ls -1 ./$year/*.gz | cut -d "_" -f 1 |sort -n | uniq -c | grep -v "^.*1 " | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
    gunzip -v $(ls -1 $duplicate* | sort -n | head -n 1)
done

In [ ]:
year = 2015
#specify start/end date for simulation
start = '2015-02-01'
end = '2015-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2015
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2016

In [ ]:
%%bash

year=2016
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2016
#specify start/end date for simulation
start = '2016-02-01'
end = '2016-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2016
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2017

In [ ]:
%%bash

year=2017
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2017
#specify start/end date for simulation
start = '2017-02-01'
end = '2017-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2017
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done

# 2018

In [ ]:
%%bash

year=2018
#decompress files
for file in $(find ./$year/ -name "*.gz")
do
    gunzip -v $file
done

In [ ]:
year = 2018
#specify start/end date for simulation
start = '2018-02-01'
end = '2018-09-30'
drift = 20 #drift time in days
#set stepsize for release in hours
step = 12

##############################################
filenames = {'U': "./"+str(year)+"/"+str(year)+"*.nc",
             'V': "./"+str(year)+"/"+str(year)+"*.nc"}

variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#################

start = np.datetime64(start+'T00:00:00.000000000')
end = np.datetime64(end+'T23:59:59')

#this is the first timepoint for which we have data in the field
first=np.datetime64(str(fieldset.U.grid.time_origin))
print ("First timepoint with data: ", first)

#this is the last timepoint for which we have data in the fieldset
last=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
print ("Last timepoint with data: ", last)

if (start >= first):
    print ("Start time is ok: ", start)
else:
    print ("Invalid start date")

if (end <= last):
    print ("End time is ok: ", end)
else:
    print ("Invalid end date")

#if the simulation needs to run for 20 days then the last day we can start it is:
#lastdata=np.datetime64(str(fieldset.U.grid.time_origin)) + np.timedelta64(int(fieldset.U.grid.time_full[-1]),'s')
laststart=end - np.timedelta64(drift, 'D')
print ("\nLast start", laststart)

#find the delta time from first data to desired starttime
releasesecond = np.timedelta64(start - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')

#at which timepoint in seconds after the origin could we start the last simulation
laststartinseconds = np.timedelta64(laststart - np.datetime64(str(fieldset.U.grid.time_origin)), 's') / np.timedelta64(1,'s')
#print (releasesecond, laststartinseconds)

##########################

### set some variables (don't change)
meta = {}
index = 0
releasesecond = int(releasesecond)
step = np.timedelta64(int(step), 'h')
pset = {}

while np.timedelta64(int(releasesecond), 's') <= np.timedelta64(int(laststartinseconds), 's'):
    meta[index] = {'seconds-post-origin': releasesecond, 'datetime': first + np.timedelta64(releasesecond, 's')}
    print ("release particle", index, 
           first + np.timedelta64(releasesecond, 's'), 
           "(", np.timedelta64(int(releasesecond), 's'), "post origin)")
    pset[index] = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=start_lon, lat=start_lat,
                             time=releasesecond)
    
    index+=1
    releasesecond = int(releasesecond + (step / np.timedelta64(1, 's')))
    
################################

#print (pset)

for i in pset.keys():
    print ("particle", i, " /", len(pset.keys()) - 1, "- released:", 
           first + np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), 
           "(", np.timedelta64(int(meta[i]['seconds-post-origin']), 's'), "post origin)")
    pset[i].execute(AdvectionRK4,
             runtime=timedelta(days=drift),
             dt=timedelta(minutes=5),
             output_file=pset[i].ParticleFile(name="./"+str(year)+"/release-"+str(meta[i]['datetime']).replace(':', '-')+".nc", 
                                           outputdt=timedelta(hours=6)))
    
###############################

del(pset)


In [ ]:
%%bash

year=2018
#compress files
for file in $(find ./$year/ -name *.nc | grep -v "release")
do
    gzip -v $file
done